In [ ]:
def calculate_percentiles(df, athlete_id, metric_dict, selected_metrics=None, verbose=False):
    """
    Calculate percentile rankings for an athlete compared to their peers, grouped by competition level.

    Args:
        df (pd.DataFrame): The dataset containing test data.
        athlete_id (int): The ID of the athlete.
        metric_dict (dict): Dictionary defining metrics to analyze.
        selected_metrics (list, optional): Specific metrics to process. Defaults to all metrics in metric_dict.
        verbose (bool): Whether to print detailed processing information.

    Returns:
        dict: Percentiles calculated for each metric.
        str: Athlete's name.
        str: Athlete's level.
    """
    # Initialize percentiles and athlete details
    percentiles = {}
    athlete_data = df[df['ID'] == athlete_id]

    # Check if athlete data exists
    if athlete_data.empty:
        if verbose:
            print(f"No data found for athlete ID {athlete_id}.")
        return {}, None, None

    # Get athlete's name and current level
    athlete_name = f"{athlete_data['First Name'].iloc[0]} {athlete_data['Last Name'].iloc[0]}"
    athlete_level = athlete_data['Level'].iloc[-1]  # Use the most recent level

    # Filter data for the athlete's current level
    peer_data = df[df['Level'] == athlete_level]
    if peer_data.empty:
        if verbose:
            print(f"No peer data found for level '{athlete_level}'.")
        return {}, athlete_name, athlete_level

    if verbose:
        print(f"Processing data for {athlete_name} at level '{athlete_level}' with {len(peer_data)} peers.")

    # Process metrics
    for (test_type, sub_type), metrics in metric_dict.items():
        for metric in metrics:
            # Skip if not in selected metrics
            if selected_metrics and metric not in selected_metrics:
                continue

            subset = peer_data[(peer_data['Test Type'] == test_type) &
                               (peer_data['Test Sub-Type'] == sub_type)]

            if metric not in subset.columns:
                if verbose:
                    print(f"Metric '{metric}' not found in data for {test_type} - {sub_type}")
                continue

            # Convert to numeric and drop NaN
            subset[metric] = pd.to_numeric(subset[metric], errors='coerce')
            subset = subset.dropna(subset=[metric])

            # Skip if subset is empty
            if subset.empty:
                if verbose:
                    print(f"No valid data for metric '{metric}' under {test_type} - {sub_type}.")
                continue

            athlete_subset = subset[subset['ID'] == athlete_id].sort_values(by='Test Number')

            # Handle cases with fewer than two tests
            if athlete_subset.empty:
                if verbose:
                    print(f"No test data available for metric '{metric}' for athlete ID {athlete_id}.")
                continue

            # Extract test values
            first_test = athlete_subset.iloc[0][metric]  # First test value
            most_recent_test = athlete_subset.iloc[-1][metric]  # Most recent test value
            previous_test = athlete_subset.iloc[-2][metric] if len(athlete_subset) > 1 else first_test

            # Ensure no NaN values are included in calculations
            first_test = first_test if pd.notna(first_test) else 0
            most_recent_test = most_recent_test if pd.notna(most_recent_test) else 0
            previous_test = previous_test if pd.notna(previous_test) else 0

            # Calculate percentile ranks
            first_percentile = percentileofscore(subset[metric], first_test)
            recent_percentile = percentileofscore(subset[metric], most_recent_test)
            previous_percentile = percentileofscore(subset[metric], previous_test)

            # Store results
            label = f"{test_type} - {sub_type} - {metric}"
            percentiles[label] = {
                'first_percentile': first_percentile,
                'previous_percentile': previous_percentile,
                'recent_percentile': recent_percentile,
                'first_value': first_test,
                'previous_value': previous_test,
                'recent_value': most_recent_test
            }

            if verbose:
                print(f"Calculated percentiles for {label}: {percentiles[label]}")

    return percentiles, athlete_name, athlete_level
